Практическое задание

Вариант 1. (простой)

    обучить сверточную нейронную сеть в стиле AlexNet (с падением размера ядра свертки и последовательностью блоков свертка-пулинг (conv-pool)-(conv-pool)-...) на датасете fashion-mnist или cifar10
    оценить рост точности при увеличении ширины сети (больше ядер)
    оценить рост точности при увеличении глубины сети (больше слоев)
    (опциционально)сравнить с точностью полносвязной сети для этой выборки


In [ ]:
from tensorflow.keras.datasets import fashion_mnist
from tensorflow.keras.utils import to_categorical

import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Conv2D, Input
from tensorflow.keras.layers import GlobalMaxPooling2D, MaxPooling2D, AveragePooling2D, GlobalMaxPooling2D
from tensorflow.keras import losses
from tensorflow.keras import layers

In [ ]:
# loading train, test data
(x_train, y_train), (x_test, y_test) = fashion_mnist.load_data()

x_train = tf.pad(x_train, [[0, 0], [2,2], [2,2]])/255
x_test = tf.pad(x_test, [[0, 0], [2,2], [2, 2]])/255

x_train = tf.expand_dims(x_train, axis=3, name=None)
x_test = tf.expand_dims(x_test, axis=3, name=None)

x_train = tf.repeat(x_train, 3, axis=3)
x_test = tf.repeat(x_test, 3, axis=3)

x_val = x_train[-2000:,:,:]
y_val = y_train[-2000:]
x_train = x_train[:-2000,:,:,:]
y_train = y_train[:-2000]
# conversation from uint8 to float32
#x_train = x_train.astype('float32')
#x_test = x_test.astype('float32')

# data normalization [0, 1]
#x_train /=255
#x_test /= 255

# one-hot encoding
#y_train = to_categorical(y_train, 10)
#y_test = to_categorical(y_test, 10)

# convercation shape to 4D
#x_train = x_train.reshape(x_train.shape[0], 28, 28, 1)
#x_test = x_test.reshape(x_test.shape[0], 28, 28, 1)

4431872/4422102 [==============================] - 0s 0us/step


In [ ]:
x_val.shape

TensorShape([2000, 32, 32, 3])

In [ ]:
#x_valid, y_valid = x_train[:5000], y_train[:5000]

In [ ]:
#x_train, y_train = x_train[5000:], y_train[5000:]

In [ ]:
#x_train.shape

(55000, 28, 28)

In [ ]:
#x_train_ds = tf.data.Dataset.from_tensor_slices((x_train, y_train))
#x_test_ds = tf.data.Dataset.from_tensor_slices((x_test, y_test))
#x_valid_ds = tf.data.Dataset.from_tensor_slices((x_valid, y_valid))

In [ ]:
#x_train_ds

<TensorSliceDataset shapes: ((28, 28), ()), types: (tf.uint8, tf.uint8)>

In [ ]:
from tensorflow.keras import layers
model = Sequential()

model.add(layers.experimental.preprocessing.Resizing(224, 224, interpolation="bilinear", input_shape=x_train.shape[1:]))

model.add(layers.Conv2D(filters = 96, kernel_size=(11,11), strides=(4,4), padding='same'))
model.add(layers.Lambda(tf.nn.local_response_normalization))
model.add(layers.Activation('relu'))
model.add(layers.MaxPooling2D(pool_size=(3,3), strides=(2,2)))

model.add(layers.Conv2D(filters=256, kernel_size=(5,5), strides=(4,4), padding='same'))
model.add(layers.Lambda(tf.nn.local_response_normalization))
model.add(layers.Activation('relu'))
model.add(layers.MaxPooling2D(pool_size=(3,3), strides=2))

model.add(layers.Conv2D(filters=384, kernel_size=(3,3), strides=(4,4), padding='same'))
model.add(layers.Activation('relu'))

model.add(layers.Conv2D(filters=384, kernel_size=(3,3), strides=(4,4), padding='same'))
model.add(layers.Activation('relu'))

model.add(layers.Conv2D(filters=256, kernel_size=(3,3), strides=(4,4), padding='same'))
model.add(layers.Activation('relu'))

model.add(layers.Flatten())

model.add(layers.Dense(4096, activation='relu'))
model.add(layers.Dropout(0.5))

model.add(layers.Dense(4096, activation='relu'))
model.add(layers.Dropout(0.5))

model.add(layers.Dense(10, activation='softmax'))

model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 resizing_3 (Resizing)       (None, 224, 224, 3)       0         
                                                                 
 conv2d_15 (Conv2D)          (None, 56, 56, 96)        34944     
                                                                 
 lambda_6 (Lambda)           (None, 56, 56, 96)        0         
                                                                 
 activation_15 (Activation)  (None, 56, 56, 96)        0         
                                                                 
 max_pooling2d_6 (MaxPooling  (None, 27, 27, 96)       0         
 2D)                                                             
                                                                 
 conv2d_16 (Conv2D)          (None, 7, 7, 256)         614656    
                                                      

In [ ]:
model.compile(optimizer='adam', loss=losses.sparse_categorical_crossentropy, metrics=['accuracy'])

history = model.fit(x_train, y_train, batch_size=64, epochs=5, validation_data=(x_val, y_val))

Epoch 1/5
907/907 [==============================] - 242s 266ms/step - loss: 0.6152 - accuracy: 0.7537 - val_loss: 0.3280 - val_accuracy: 0.8800
Epoch 2/5
907/907 [==============================] - 238s 262ms/step - loss: 0.3371 - accuracy: 0.8769 - val_loss: 0.2819 - val_accuracy: 0.8905
Epoch 3/5
907/907 [==============================] - 238s 262ms/step - loss: 0.2889 - accuracy: 0.8958 - val_loss: 0.2712 - val_accuracy: 0.8990
Epoch 4/5
907/907 [==============================] - 238s 262ms/step - loss: 0.2589 - accuracy: 0.9067 - val_loss: 0.2810 - val_accuracy: 0.9035
Epoch 5/5
907/907 [==============================] - 240s 264ms/step - loss: 0.2429 - accuracy: 0.9125 - val_loss: 0.2539 - val_accuracy: 0.9075


In [ ]:
test_loss, test_accuracy = model.evaluate(x_test, y_test)
print(test_loss, test_accuracy)

313/313 [==============================] - 19s 60ms/step - loss: 0.2860 - accuracy: 0.8998
0.2860470712184906 0.8998000025749207


2. Оценить рост точности при увеличении ширины сети (больше ядер)

In [ ]:
from tensorflow.keras import layers
model = Sequential()

model.add(layers.experimental.preprocessing.Resizing(224, 224, interpolation="bilinear", input_shape=x_train.shape[1:]))

model.add(layers.Conv2D(filters = 150, kernel_size=(11,11), strides=(4,4), padding='same'))
model.add(layers.Lambda(tf.nn.local_response_normalization))
model.add(layers.Activation('relu'))
model.add(layers.MaxPooling2D(pool_size=(3,3), strides=(2,2)))

model.add(layers.Conv2D(filters=300, kernel_size=(5,5), strides=(4,4), padding='same'))
model.add(layers.Lambda(tf.nn.local_response_normalization))
model.add(layers.Activation('relu'))
model.add(layers.MaxPooling2D(pool_size=(3,3), strides=2))

model.add(layers.Conv2D(filters=450, kernel_size=(3,3), strides=(4,4), padding='same'))
model.add(layers.Activation('relu'))

model.add(layers.Conv2D(filters=450, kernel_size=(3,3), strides=(4,4), padding='same'))
model.add(layers.Activation('relu'))

model.add(layers.Conv2D(filters=300, kernel_size=(3,3), strides=(4,4), padding='same'))
model.add(layers.Activation('relu'))

model.add(layers.Flatten())

model.add(layers.Dense(4096, activation='relu'))
model.add(layers.Dropout(0.5))

model.add(layers.Dense(4096, activation='relu'))
model.add(layers.Dropout(0.5))

model.add(layers.Dense(10, activation='softmax'))

model.summary()

Model: "sequential_17"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 resizing_17 (Resizing)      (None, 224, 224, 3)       0         
                                                                 
 conv2d_53 (Conv2D)          (None, 56, 56, 150)       54600     
                                                                 
 lambda_50 (Lambda)          (None, 56, 56, 150)       0         
                                                                 
 activation_53 (Activation)  (None, 56, 56, 150)       0         
                                                                 
 max_pooling2d_47 (MaxPoolin  (None, 27, 27, 150)      0         
 g2D)                                                            
                                                                 
 conv2d_54 (Conv2D)          (None, 7, 7, 300)         1125300   
                                                     

In [ ]:
model.compile(optimizer='adam', loss=losses.sparse_categorical_crossentropy, metrics=['accuracy'])

history = model.fit(x_train, y_train, batch_size=64, epochs=5, validation_data=(x_val, y_val))

Epoch 1/5
907/907 [==============================] - 277s 293ms/step - loss: 0.6880 - accuracy: 0.7266 - val_loss: 0.3475 - val_accuracy: 0.8695
Epoch 2/5
907/907 [==============================] - 267s 294ms/step - loss: 0.3442 - accuracy: 0.8765 - val_loss: 0.3326 - val_accuracy: 0.8725
Epoch 3/5
907/907 [==============================] - 265s 292ms/step - loss: 0.2970 - accuracy: 0.8942 - val_loss: 0.2797 - val_accuracy: 0.8945
Epoch 4/5
907/907 [==============================] - 265s 292ms/step - loss: 0.2742 - accuracy: 0.9020 - val_loss: 0.2687 - val_accuracy: 0.9085
Epoch 5/5
907/907 [==============================] - 265s 293ms/step - loss: 0.2487 - accuracy: 0.9111 - val_loss: 0.2612 - val_accuracy: 0.9085


In [ ]:
test_loss, test_accuracy = model.evaluate(x_test, y_test)
print(test_loss, test_accuracy)

313/313 [==============================] - 20s 64ms/step - loss: 0.2856 - accuracy: 0.9031
0.2855660319328308 0.9031000137329102


###Вывод: С увеличением числа фильтров растет точность модели.

3. Оценить рост точности при увеличении глубины сети (больше слоев)

In [ ]:
from tensorflow.keras import layers
model = Sequential()

model.add(layers.experimental.preprocessing.Resizing(224, 224, interpolation="bilinear", input_shape=x_train.shape[1:]))

model.add(layers.Conv2D(filters = 96, kernel_size=(22,22), strides=(4,4), padding='same'))
model.add(layers.Lambda(tf.nn.local_response_normalization))
model.add(layers.Activation('relu'))
model.add(layers.MaxPooling2D(pool_size=(3,3), strides=(2,2)))

model.add(layers.Conv2D(filters=256, kernel_size=(10,10), strides=(4,4), padding='same'))
model.add(layers.Lambda(tf.nn.local_response_normalization))
model.add(layers.Activation('relu'))
model.add(layers.MaxPooling2D(pool_size=(3,3), strides=2))

model.add(layers.Conv2D(filters=384, kernel_size=(9,9), strides=(4,4), padding='same'))
model.add(layers.Activation('relu'))

model.add(layers.Conv2D(filters=384, kernel_size=(9,9), strides=(4,4), padding='same'))
model.add(layers.Activation('relu'))

model.add(layers.Conv2D(filters=256, kernel_size=(9,9), strides=(4,4), padding='same'))
model.add(layers.Activation('relu'))

model.add(layers.Flatten())

model.add(layers.Dense(4096, activation='relu'))
model.add(layers.Dropout(0.5))

model.add(layers.Dense(4096, activation='relu'))
model.add(layers.Dropout(0.5))

model.add(layers.Dense(10, activation='softmax'))

model.summary()

Model: "sequential_23"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 resizing_23 (Resizing)      (None, 224, 224, 3)       0         
                                                                 
 conv2d_83 (Conv2D)          (None, 56, 56, 96)        139488    
                                                                 
 lambda_62 (Lambda)          (None, 56, 56, 96)        0         
                                                                 
 activation_83 (Activation)  (None, 56, 56, 96)        0         
                                                                 
 max_pooling2d_59 (MaxPoolin  (None, 27, 27, 96)       0         
 g2D)                                                            
                                                                 
 conv2d_84 (Conv2D)          (None, 7, 7, 256)         2457856   
                                                     

In [ ]:
model.compile(optimizer='adam', loss=losses.sparse_categorical_crossentropy, metrics=['accuracy'])

history = model.fit(x_train, y_train, batch_size=64, epochs=5, validation_data=(x_val, y_val))

Epoch 1/5
907/907 [==============================] - 348s 381ms/step - loss: 0.7083 - accuracy: 0.7191 - val_loss: 0.3696 - val_accuracy: 0.8665
Epoch 2/5
907/907 [==============================] - 344s 380ms/step - loss: 0.3264 - accuracy: 0.8857 - val_loss: 0.2808 - val_accuracy: 0.9035
Epoch 3/5
907/907 [==============================] - 344s 380ms/step - loss: 0.2761 - accuracy: 0.9021 - val_loss: 0.2587 - val_accuracy: 0.9040
Epoch 4/5
907/907 [==============================] - 344s 380ms/step - loss: 0.2438 - accuracy: 0.9132 - val_loss: 0.2603 - val_accuracy: 0.9065
Epoch 5/5
907/907 [==============================] - 344s 380ms/step - loss: 0.2209 - accuracy: 0.9210 - val_loss: 0.2354 - val_accuracy: 0.9185


In [ ]:
test_loss, test_accuracy = model.evaluate(x_test, y_test)
print(test_loss, test_accuracy)

313/313 [==============================] - 29s 92ms/step - loss: 0.2745 - accuracy: 0.9087
0.27452969551086426 0.9086999893188477


### Вывод: С увеличением размера ядра растет точность модели, но это может привести к переобучению.